In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from collections import namedtuple
from bs4 import BeautifulSoup
import re
from dateutil import parser

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://www.pcdf.df.gov.br/servicos/desaparecidos?start=0'
driver.get(website)

driver.maximize_window()

In [10]:
# Definindo a estrutura da tupla com os nomes dos campos
Registro = namedtuple('Registro', 'nome data_desaparecimento região_desaparecimento idade')

registros = []

try:
    # Processamento da primeira página
    time.sleep(5)  # Aguarde a página carregar
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    for li in soup.find_all('li', class_='odd'):
        nome = li.find('a', class_='moduleItemTitle').text.strip() if li.find('a', class_='moduleItemTitle') else ''
        p_elements = li.find_all('p')
        data_desaparecimento = p_elements[0].text.strip() if len(p_elements) > 0 else ''
        região_desaparecimento = p_elements[1].text.strip() if len(p_elements) > 1 else ''
        idade = p_elements[2].text.strip() if len(p_elements) > 2 else ''
        registro = Registro(nome, data_desaparecimento, região_desaparecimento, idade)
        registros.append(registro)

    # Clicar no botão "Próxima página" se disponível
    next_page_button = driver.find_elements(By.CSS_SELECTOR, 'a.hasTooltip.pagenav')
    if next_page_button:
        next_page_button[0].click()
        time.sleep(5)  # Aguarde a segunda página carregar

        # Processamento da segunda página
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        for li in soup.find_all('li', class_='odd'):
            nome = li.find('a', class_='moduleItemTitle').text.strip() if li.find('a', class_='moduleItemTitle') else ''
            p_elements = li.find_all('p')
            data_desaparecimento = p_elements[0].text.strip() if len(p_elements) > 0 else ''
            região_desaparecimento = p_elements[1].text.strip() if len(p_elements) > 1 else ''
            idade = p_elements[2].text.strip() if len(p_elements) > 2 else ''
            registro = Registro(nome, data_desaparecimento, região_desaparecimento, idade)
            registros.append(registro)

finally:
    driver.quit()  # Certifique-se de que o navegador é fechado corretamente

# Exibindo os resultados
for r in registros:
    print(r)

Registro(nome='DIRLEY DE SOUZA OLIVEIRA -  DESAPARECIDO QUANDO ESTAVA COM 16 ANOS', data_desaparecimento='DATA DE DESAPARECIMENTO: 30/09/2005, ESTAVA COM 16 ANOS.', região_desaparecimento='REGIÃO DO DESAPARECIMENTO: ÁREA DE DESENVOLVIMENTO ECONÔMICO EM ÁGUAS CLARAS.', idade='PROGRESSÃO DE IDADE ATUAL: 2020, COM\xa0 31 ANOS.')
Registro(nome='YANARA LIMA GOMES - DESAPARECIDA COM 15 ANOS', data_desaparecimento='Data de desaparecimento: 20/02/2016', região_desaparecimento='Data de nascimento:\xa003/07/2000', idade='Progressão de idade mais atual: feita em 2021, com 20 anos')
Registro(nome='LEUZENILDA MARQUES DA ROCHA - Desaparecida com 14 anos de idade', data_desaparecimento='Data de desaparecimento:10 de junho de 2007.', região_desaparecimento='Região: Paranoá/ DF', idade='Progressão de idade mais atual: 2019, com 28 anos.')
Registro(nome='DANDARA GONÇALVES XAVIER - Desaparecida com 15 anos', data_desaparecimento='Data de desaparecimento:\xa026 de agosto de 2003, com 15 anos.', região_des

In [19]:
def limpar_nome(nome):
    # Usa regex para dividir o nome nos delimitadores " - " ou ","
    partes = re.split(r' - |,', nome)
    return partes[0].strip()

registros_limpos = []
for registro in registros:
    nome_limpo = limpar_nome(registro.nome)  # Limpa o nome com a nova função
    # Cria uma nova namedtuple com o nome limpo e os outros campos originais
    registro_limpo = Registro(nome=nome_limpo, data_desaparecimento=registro.data_desaparecimento, região_desaparecimento=registro.região_desaparecimento, idade=registro.idade)
    registros_limpos.append(registro_limpo)

# Imprimindo os registros limpos para verificar
for registro in registros_limpos:
    print(registro)

Registro(nome='DIRLEY DE SOUZA OLIVEIRA', data_desaparecimento='DATA DE DESAPARECIMENTO: 30/09/2005, ESTAVA COM 16 ANOS.', região_desaparecimento='REGIÃO DO DESAPARECIMENTO: ÁREA DE DESENVOLVIMENTO ECONÔMICO EM ÁGUAS CLARAS.', idade='PROGRESSÃO DE IDADE ATUAL: 2020, COM\xa0 31 ANOS.')
Registro(nome='YANARA LIMA GOMES', data_desaparecimento='Data de desaparecimento: 20/02/2016', região_desaparecimento='Data de nascimento:\xa003/07/2000', idade='Progressão de idade mais atual: feita em 2021, com 20 anos')
Registro(nome='LEUZENILDA MARQUES DA ROCHA', data_desaparecimento='Data de desaparecimento:10 de junho de 2007.', região_desaparecimento='Região: Paranoá/ DF', idade='Progressão de idade mais atual: 2019, com 28 anos.')
Registro(nome='DANDARA GONÇALVES XAVIER', data_desaparecimento='Data de desaparecimento:\xa026 de agosto de 2003, com 15 anos.', região_desaparecimento='Região: Estrutural', idade='Progressão de idade mais atual:\xa02019, com 31 anos.')
Registro(nome='ANDREZA CARLA DE LI

In [21]:
registro_limpo = namedtuple('Registro', 'nome detalhes')

registros_reformulados = []
for registro in registros_limpos:
    # Concatenando as informações em uma única string
    detalhes = f"{registro.data_desaparecimento}, {registro.região_desaparecimento}, {registro.idade}"
    # Criando um novo registro com nome e detalhes
    registro_reformulado = Registro(nome=registro.nome, detalhes=detalhes)
    registros_reformulados.append(registro_reformulado)

# Imprimindo os registros reformulados para verificar
for registro in registros_reformulados:
    print(registro)

Registro(nome='DIRLEY DE SOUZA OLIVEIRA', detalhes='DATA DE DESAPARECIMENTO: 30/09/2005, ESTAVA COM 16 ANOS., REGIÃO DO DESAPARECIMENTO: ÁREA DE DESENVOLVIMENTO ECONÔMICO EM ÁGUAS CLARAS., PROGRESSÃO DE IDADE ATUAL: 2020, COM\xa0 31 ANOS.')
Registro(nome='YANARA LIMA GOMES', detalhes='Data de desaparecimento: 20/02/2016, Data de nascimento:\xa003/07/2000, Progressão de idade mais atual: feita em 2021, com 20 anos')
Registro(nome='LEUZENILDA MARQUES DA ROCHA', detalhes='Data de desaparecimento:10 de junho de 2007., Região: Paranoá/ DF, Progressão de idade mais atual: 2019, com 28 anos.')
Registro(nome='DANDARA GONÇALVES XAVIER', detalhes='Data de desaparecimento:\xa026 de agosto de 2003, com 15 anos., Região: Estrutural, Progressão de idade mais atual:\xa02019, com 31 anos.')
Registro(nome='ANDREZA CARLA DE LIMA DO NASCIMENTO', detalhes='Data de desaparecimento: 09/11/2008, Região: Ceilândia/DF., Progressão de idade mais atual: 2013, com 18 anos.')
Registro(nome='Paulo Henrique Alves Gu

In [23]:
df = pd.DataFrame(registros_reformulados)
df.to_csv('Desaparecidos Polícia Civil DF.csv', index=False)